# Scikit Learn

Miguel Ángel Jiménez Cuadrillero
7/5/2020

## Creación y evaluación de modelos predictivos en python con Scikit-learn


### ¿Cómo instalar scikit-learn?

pip install scikit-learn


## Cómo crear un modelo simple

Para este ejemplo utilizaremos el archiconocido iris dataset y un tipo de modelo simple con un único hiperparámetro: usaremos un clasificador k-neighbours con n_neighbors=1. Este es un modelo muy simple e intuitivo que dice "la etiqueta de un punto desconocido es la misma que la etiqueta de su punto de entrenamiento más cercano"

## Cargar Dataset


Información del conjunto de datos:

Esta es quizás la base de datos más conocida que se encuentra en la literatura de Machine Learning. El conjunto de datos contiene 3 clases de 50 instancias cada una, donde cada clase se refiere a un tipo de planta de iris. Una clase es linealmente separable de las otras 2; estos últimos NO son linealmente separables entre sí.

Atributo previsto: clase de planta de iris.

Información de los atributos:

* longitud del sépalo en cm
* ancho del sépalo en cm
* longitud del pétalo en cm
* ancho del pétalo en cm
* clase:
    - Iris Setosa
    - Iris Versicolour
    - Iris Virginica

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

Para crear un modelo simple, en primer lugar debemos saber si es de clasificación o de regresión y con esa información buscar la clase que represente al tipo de modelo en la siguiente API:

https://scikit-learn.org/stable/modules/classes.html


En este caso buscamos el clasificador KNeighborsClassifier

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

In [2]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1) #Configuramos el hiperparámetro para tener la línea base.

In [3]:
#Entrenamos el modelo con los datos de entrenamiento
model.fit(X, y)

KNeighborsClassifier(n_neighbors=1)

In [4]:
#Realizamos predicciones en base a los datos de entrenamiento.
y_model = model.predict(X)

In [5]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_model)

1.0

El modelo precisión de 1.0, lo que indica que nuestro modelo etiquetó correctamente el 100% de los puntos

¿Pero esto realmente mide la precisión esperada? ¿Realmente hemos encontrado un modelo que esperamos sea correcto el 100% las veces?

La respuesta es no y es por un error fundamental: se entrena y evalúa el modelo con los mismos datos . Además, el modelo K-nearest neigbours es un estimador basado en instancias que simplemente almacena los datos de entrenamiento y predice las etiquetas comparando datos nuevos con estos puntos almacenados: siempre dará 100% si consultamos por los datos con los que entrenamos.

## Cómo usar la validación cruzada o Cross Validation para evitar el sobreajuste

### Validación del modelo mediante Holdout sets

Para tener una mejor idea de lo bueno que es el modelo, se utilizan los datos de validación, es decir, hold-out set: retenemos un subconjunto de los datos del entrenamiento del modelo y luego usamos este conjunto para verificar el rendimiento del modelo. 

Esta división se puede hacer usando la train_test_splitutilidad en Scikit-Learn:

In [6]:
from sklearn.model_selection import train_test_split
# split the data with 50% in each set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
model.fit(X_train, y_train)

# evaluate the model on the second set of data
y_predicted = model.predict(X_test)
accuracy_score(y_test, y_predicted)

0.9066666666666666

Vemos aquí un resultado más razonable: el clasificador del vecino más cercano tiene una precisión del 90% en este  hold-out set. El conjunto de retención es similar a datos desconocidos, porque el modelo no lo ha "visto" antes.

## Validación del modelo mediante Cross-Validation


Una desventaja de usar un conjunto de reserva para la validación del modelo es que hemos perdido una parte de nuestros datos en el entrenamiento del modelo. En el caso anterior, ¡la mitad del conjunto de datos no contribuye a la capacitación del modelo! Esto no es óptimo y puede causar problemas, especialmente si el conjunto inicial de datos de entrenamiento es pequeño (como es el caso)

Una forma de abordar esto es utilizar la validación cruzada ; es decir, hacer una secuencia de ajustes donde cada subconjunto de datos se usa como un conjunto de entrenamiento y como un conjunto de validación.

Vamos a hacer una prueba usando la mitad de los datos como entrenamiento y como test alternativamente:

In [7]:
y2_model = model.fit(X_train, y_train).predict(X_test)
y1_model = model.fit(X_test, y_test).predict(X_train)
accuracy_score(y_train, y1_model), accuracy_score(y_test, y2_model)

(0.96, 0.9066666666666666)

Lo que sale son dos puntajes de precisión distintos, que podríamos combinar (por ejemplo, tomando la media) para obtener una mejor medición del rendimiento del modelo global. Esta forma particular de validación cruzada es una validación cruzada doble, es decir, una en la que hemos dividido los datos en dos conjuntos y los hemos utilizado a su vez como un conjunto de validación y entrenamiento.

Podríamos ampliar esta idea para usar aún más pruebas y más pliegues en los datos --> K-fold Cross Validation

Scikit Learn ya tiene mecanismos para este tipo de entrenamiento y estimación de prestaciones: *cross_val_score*

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

Este método nos dará la evaluación de la métrica que definamos, y en el caso de no estar definida, será la que tenga por defecto el estimador. Al ser un modelo de clasificación la métrica por defecto es *accuracy*

Otras métricas:

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

*cross_validate* es similar pero permite evaluar varias métricas:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate


In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

La repetición de la validación en diferentes subconjuntos de datos nos da una idea aún mejor del rendimiento del algoritmo.

Scikit-Learn implementa una serie de esquemas útiles de validación cruzada que son útiles en situaciones particulares; Estos se implementan a través de iteradores en el módulo *"sklearn.model_selection"* (antoguo cross_validation). 

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection

Por ejemplo, podríamos desear llegar al caso extremo en el que nuestro número de pliegues es igual al número de puntos de datos: es decir, entrenamos en todos los puntos menos uno en cada prueba. Este tipo de validación cruzada se conoce como validación cruzada LeaveOneOut y se puede utilizar de la siguiente manera:

In [9]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
#media de las puntuaciones:

scores.mean()

0.96

## Métricas

Una gran parte del trabajo de un data scientist es evaluar modelos de forma correcta. En el siguiente tenemos una explicación de las métricas que podemos utilizar para evaluar el performance de los modelos que creemos.

https://scikit-learn.org/stable/modules/model_evaluation.html

## ¿Cómo encontrar los mejores parámetros para su modelo elegido?

Un punto importante es que una clase de modelo no es lo mismo que una instancia de un modelo.

Una vez que hayamos decidido nuestra clase de modelo, todavía hay algunas opciones disponibles que tenemos que configurar. Dependiendo de la clase de modelo con la que estamos trabajando, es posible que necesitemos responder una o más preguntas como las siguientes:

* ¿Nos gustaría ajustar el desplazamiento (es decir, la intersección con el eje y)?
* ¿Nos gustaría que el modelo se normalice?
* ¿Nos gustaría preprocesar nuestras funciones para agregar flexibilidad al modelo?
* ¿Qué grado de regularización nos gustaría utilizar en nuestro modelo?
* ...


Estos son ejemplos de las elecciones importantes que deben hacerse una vez que se selecciona la clase de modelo. Estas opciones a menudo se representan como hiperparámetros o parámetros que deben establecerse antes de que el modelo se ajuste a los datos.

En Scikit-Learn, los hiperparámetros se eligen pasando valores en la instanciación del modelo. 

En nuestro caso el hiperparámetro que podemos configurar es valor de K, vamos a hacer un barrido sobre los datos mediante cross validation y variando el valor de K para encontrar la mejor combinación (en este caso sólo un valor) de hiperparámetros.

In [11]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'n_neighbors': [1,2,3,4,5,6,7]}

grid = GridSearchCV(model, param_grid, cv=5)

In [12]:
grid.fit(X, y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_neighbors=1),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7]})

In [13]:
grid.best_params_

{'n_neighbors': 6}

In [14]:
#utilizamos el mejor modelo
y_pred = grid.predict(X_test)

In [15]:
from sklearn import metrics

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.96
